In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import numpy as np
import math
import torch.optim as optim

In [2]:
d = {
    "good": 0,
    "neutral": 1,
    "bad" : 2
    
}
def encoding(label):
    return d[label]

In [ ]:
class RecurrentNetworkWithSleepData(nn.Module):
    def __init__(self, seq_length, hidden_size, num_layers):
        super(RecurrentNetwork, self).__init__()
        self.rnn = nn.RNN(input_size=7, hidden_size=hidden_size, num_layers=num_layers, batch_first=True,  nonlinearity='relu')
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(seq_length*hidden_size + 17,3),           
        )
    
    def forward(self, x):
        encoded_actigraphy_features, _ = self.rnn(x[:, :, :7])#actigraphy features
        sleep_features = x[:, :, 7:]
        concatenated = torch.cat((encoded_actigraphy_features, sleep_features), dim=2)
        return self.classifier(concatenated)